# Grupo
1. Arthur Querido Lopes                            11711ECP014
2. Felipe Junio Rezende                            11711ECP007
3. Marina Candeo Medeiros e Silva                  11621ECP006


### Importação das bibliotecas

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact_manual, fixed
import scipy.signal
from IPython.display import clear_output

### Definição do kernel (filtros)

In [2]:
kernel = {
    'identity': np.array([[0,0,0],[0,1,0],[0,0,0]], dtype=float),
    'edge detection': np.array([[1,0,-1],[0,0,0],[-1,0,1]], dtype=float),
    'laplacian': np.array([[0,-1,0],[-1,4,-1],[0,-1,0]], dtype=float),
    'laplacian w/ diagonals': np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]], dtype=float),
    'laplacian of gaussian': np.array([[0,0,-1,0,0],[0,-1,-2,-1,0],[-1,-2,16,-2,-1],[0,-1,-2,-1,0],[0,0,-1,0,0]], dtype=float),
    'scharr': np.array([[-3, 0, 3],[-10,0,10],[-3, 0, 3]], dtype=float),
    'sobel edge horizontal': np.array([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=float),
    'sobel edge vertical': np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=float),
    'line detection horizontal': np.array([[-1,-1,-1],[2,2,2],[-1,-1,-1]], dtype=float),
    'line detection vertical': np.array([[-1,2,-1],[-1,2,-1],[-1,2,-1]], dtype=float),
    'line detection 45°': np.array([[-1,-1,2],[-1,2,-1],[2,-1,-1]], dtype=float),
    'line detection 135°': np.array([[2,-1,-1],[-1,2,-1],[-1,-1,2]], dtype=float),
    'box blur': (1/9)*np.ones((3,3), dtype=float),
    'gaussian blur 3x3': (1/16)*np.array([[1,2,1],[2,4,2],[1,2,1]], dtype=float),
    'gaussian blur 5x5': (1/256)*np.array([[1,4,6,4,1],[4,16,24,16,4],[6,24,36,24,6],[4,16,24,16,4],[1,4,6,4,1]], dtype=float),
    'sharpen': np.array([[0,-1,0],[-1,5,-1],[0,-1,0]], dtype=float),
    'unsharp masking': (-1/256)*np.array([[1,4,6,4,1],[4,16,24,16,4],[6,24,-476,24,6],[4,16,24,16,4],[1,4,6,4,1]], dtype=float),
}

### Implementação dos filtros e do detecção de blur

In [3]:
def detect_blur_fft(image, thresh, show_magnitude, size=50):
    h, w = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))

    spec = np.fft.fft2(image)
    specShift = np.fft.fftshift(spec) 
    specShift[cY - size:cY + size, cX - size:cX + size] = 0
    specShift = np.fft.ifftshift(specShift)
    img_reconstructed = np.fft.ifft2(specShift)
    
    magnitude = 20 * np.log(np.abs(img_reconstructed))
    mean = np.mean(magnitude)
    
    if show_magnitude:
        plt.figure(figsize=(6,6))
        plt.imshow(magnitude, cmap='plasma')
        plt.show()
        clear_output(wait=True)
    return (mean, mean <= thresh)

def blur_and_filters(filtro, show_magnitude, threshold = 9):
    cap = cv2.VideoCapture(0)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            conv = np.uint8(np.round(scipy.signal.convolve2d(gray, kernel[filtro], boundary='fill', fillvalue =0)))
            (mean, blurry) = detect_blur_fft(gray, threshold, show_magnitude)
            
            cv2.putText(conv, str(round(mean,2)), (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            if blurry:
                cv2.putText(conv, ' Blurry', (120, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow('Frame', conv)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()
    
interact_manual(blur_and_filters, filtro=list(kernel.keys()), threshold=(5,18,1), show_magnitude=False)


interactive(children=(Dropdown(description='filtro', options=('identity', 'edge detection', 'laplacian', 'lapl…

<function __main__.blur_and_filters(filtro, show_magnitude, threshold=9)>